# Preparación del entorno y los datos

## Preparación del entorno

In [ ]:
# Instalar las bibliotecas necesarias
!pip install transformers
!pip install sentence_transformers
!pip install torch
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## Importación de bibliotecas

In [ ]:
# Importar bibliotecas de manipulación de datos y visualización
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

# Importar bibliotecas de procesamiento del lenguaje natural (NLP)
import nltk
from nltk.corpus import stopwords

# Importar bibliotecas de machine learning y procesamiento de texto
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score

# Importar bibliotecas de transformers y pytorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

# Importar biblioteca para trabajar con Google Drive
from google.colab import drive

## Configuración de Google Drive

In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Definición de rutas a los datos

In [ ]:
# Definir rutas a la carpeta de datos
#data_dir = '/Documents/escuela/tecMonterrey/semestre8/Ciencias/Reto'
data_dir = '/content/drive/MyDrive/RetoDesarrollo/dataset/'
# Ruta a datos que nosotros creamos
# clasification_data_dir = os.path.join(data_dir, 'plagiarism')

# Ruta a datos otorgados por los profesores
construction_data_dir = os.path.join(data_dir, 'train')
test_data_dir = os.path.join(data_dir, 'test')
clasification_data_dir = os.path.join(data_dir, 'plagiarism')

#clasification_data_dir = 'C:\\Users\\nadia\\Documents\\escuela\\tecMonterrey\\semestre8\\Ciencias\\Reto\\plagiarism'
#construction_data_dir = 'C:\\Users\\nadia\\Documents\\escuela\\tecMonterrey\\semestre8\\Ciencias\\Reto\\train'
#test_data_dir = 'C:\\Users\\nadia\\Documents\\escuela\\tecMonterrey\\semestre8\\Ciencias\\Reto\\test'

In [ ]:
# Listar archivos de plagio
plagiarism_files = sorted([os.path.join(clasification_data_dir, f) for f in os.listdir(clasification_data_dir)])

# Listar archivos de construcción y prueba
construction_files = sorted([os.path.join(construction_data_dir, f) for f in os.listdir(construction_data_dir)])
test_files = sorted([os.path.join(test_data_dir, f) for f in os.listdir(test_data_dir)])

## Lectura de archivos

In [ ]:
# Función para leer archivos
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Leer todos los archivos de construcción plagiados
plagiarism_texts = [read_file(f) for f in plagiarism_files]

# Leer todos los archivos de construcción
construction_texts = [read_file(f) for f in construction_files]

# Leer todos los archivos de prueba
test_texts = [read_file(f) for f in test_files]

# Impresion de los datos contenidos
print(f'Plagiarism: {plagiarism_texts}')
print(f'Construction: {construction_texts}')
print(f'Test: {test_texts}')

Plagiarism: ['Recent developments in Artificial Intelligence (AI) have generated great expectations for the future impact of AI in education and learning (AIED). Often these expectations have been influenced by misconceptions about current technical capabilities, insufficient awareness of the latest advancements in AI for education, and overly limited perspectives on the roles of education in society. In this article, we provide a review of existing AI systems in education and their pedagogic and educational assumptions. We create a classification of AIED systems and outline various approaches to incorporating AI in education and learning, demonstrating how these are based on different understandings of what AI and education are or could become, and highlight some potential challenges on the path to AIED implementation.', "Artificial intelligence (AI) is evolving and its application is expanding rapidly, becoming an integral part of our daily lives. In fact, AI has transformed the way 

## Limpieza de datos

In [ ]:
# Descargar y definir stopwords de nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Función para limpiar textos
def clean_text(texts):
    cleaned_texts = []
    for text in texts:
        text_lower = text.lower()
        text_cleaned = re.sub(r'[^\w\s]', '', text_lower)
        words = text_cleaned.split()
        filtered_words = [word for word in words if word not in stop_words]
        cleaned_text = ' '.join(filtered_words)
        cleaned_texts.append(cleaned_text)

    return cleaned_texts

In [ ]:
# Limpiar textos de plagio
plagiarism_clean = clean_text(plagiarism_texts)

# Limpiar textos de construcción y prueba
construction_clean = clean_text(construction_texts)
test_clean = clean_text(test_texts)

# Impresion de los datos limpios
print(f'Plagiarism: {plagiarism_clean}')
print(f'Construction: {construction_clean}')
print(f'Test: {test_clean}')

Plagiarism: ['recent developments artificial intelligence ai generated great expectations future impact ai education learning aied often expectations influenced misconceptions current technical capabilities insufficient awareness latest advancements ai education overly limited perspectives roles education society article provide review existing ai systems education pedagogic educational assumptions create classification aied systems outline various approaches incorporating ai education learning demonstrating based different understandings ai education could become highlight potential challenges path aied implementation', 'artificial intelligence ai evolving application expanding rapidly becoming integral part daily lives fact ai transformed way people learn however integration educational sector faced numerous challenges ethical concerns purpose study examine opportunities benefits obstacles ai education comprehensive review relevant literature conducted using systematic review method 

# Parte 1: Clasificación de "Plagio" o "No Plagio"

## Inicialización de modelo y tokenizador BERT

In [ ]:
# Inicializar el tokenizador y el modelo de BERT
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

## Tokenización de textos

In [ ]:
# Función para inicializar un diccionario de tokens
def init_dictionary(sent):
  # Inicializar diccionario: almacenar oraciones tokenizadas
  token = {'input_ids': [], 'attention_mask': []}
  for sentence in sent:
      # Codificar cada oración y agregar al diccionario
      new_token = tokenizer.encode_plus(sentence, max_length=128,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      token['input_ids'].append(new_token['input_ids'][0])
      token['attention_mask'].append(new_token['attention_mask'][0])
  return token

In [ ]:
# Tokenizar los textos de construcción y prueba
plagiarism_token = init_dictionary(plagiarism_clean)
construction_token = init_dictionary(construction_clean)
test_token = init_dictionary(test_clean)

# Impresion de la tokenizacion
print(f'Plagiarism: {plagiarism_token}')
print(f'Construction: {construction_token}')
print(f'Test: {test_token}')

Plagiarism: {'input_ids': [tensor([  101,  3522,  8973,  7976,  4454,  9932,  7013,  2307, 10908,  2925,
         4254,  9932,  2495,  4083,  9932,  2098,  2411, 10908,  5105, 28616,
         8663, 24422,  2015,  2783,  4087,  9859, 13990,  7073,  6745, 12607,
         2015,  9932,  2495, 15241,  3132, 15251,  4395,  2495,  2554,  3720,
         3073,  3319,  4493,  9932,  3001,  2495, 21877,  2850,  3995, 12863,
         4547, 17568,  3443,  5579,  9932,  2098,  3001, 12685,  2536,  8107,
        13543,  9932,  2495,  4083, 14313,  2241,  2367,  4824,  2015,  9932,
         2495,  2071,  2468, 12944,  4022,  7860,  4130,  9932,  2098,  7375,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  

## Reformateo de tensores de tokens

In [ ]:
# Función para reformatear los tensores de tokens
def reformat_tesnsors(token):
  # Reformatear lista de tensores a un solo tensor
  token['input_ids'] = torch.stack(token['input_ids'])
  token['attention_mask'] = torch.stack(token['attention_mask'])
  # Procesar tokens a través del modelo
  output = model(**token)
  output.keys()

  return output

In [ ]:
# Obtener salidas del modelo para los tokens de plagio, construcción y prueba
plag_output = reformat_tesnsors(plagiarism_token)
cons_output = reformat_tesnsors(construction_token)
test_output = reformat_tesnsors(test_token)

In [ ]:
# Los vectores densos de las representaciones de texto están en el tensor 'last_hidden_state'
plag_embeddings = plag_output.last_hidden_state
cons_embeddings = cons_output.last_hidden_state
test_embeddings = test_output.last_hidden_state

# Impresion de los embeddings
print(f'Plagiarism: {plag_embeddings}')
print(f'Construction: {cons_embeddings}')
print(f'Test: {test_embeddings}')

## Aplicación de máscaras a los embeddings

In [ ]:
# Función para aplicar máscaras a los embeddings
def apply_masks(token_dict, embeddings):
    att_mask = token_dict['attention_mask']
    mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
    mask_embeddings = embeddings * mask
    return mask_embeddings, mask

In [ ]:
# Obtener máscaras para los tokens de plagio, construcción y prueba
mask_plag_embeddings, mask_plag = apply_masks(plagiarism_token, plag_embeddings)
mask_cons_embeddings, mask_cons = apply_masks(construction_token, cons_embeddings)
mask_test_embeddings, mask_test = apply_masks(test_token, test_embeddings)

# Impresion de los tamaños de las máscaras
print(f'Plagiarism: {mask_plag_embeddings.shape}')
print(f'Construction: {mask_cons_embeddings.shape}')
print(f'Test: {mask_test_embeddings.shape}')

## Promedio de los embeddings

In [ ]:
# Función para promediar los embeddings
def mean_pool_embeddings(masked_embeddings, mask):
    summed_embeddings = torch.sum(masked_embeddings, 1)
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed_embeddings / summed_mask
    return mean_pooled

In [ ]:
# Obtener los promedios de los embeddings de plagio, construcción y prueba
plag_mean_pooled = mean_pool_embeddings(mask_plag_embeddings, mask_plag)
cons_mean_pooled = mean_pool_embeddings(mask_cons_embeddings, mask_cons)
test_mean_pooled = mean_pool_embeddings(mask_test_embeddings, mask_test)

# Impresion de los promedios de los embeddings
print(f'Plagiarism: {plag_mean_pooled}')
print(f'Construction: {cons_mean_pooled}')
print(f'Test: {test_mean_pooled}')

## Cálculo final de similitud de coseno

In [ ]:
# Convertir del tensor de PyTorch a una matriz numpy
plag_mean_pooled = plag_mean_pooled.detach().numpy()
cons_mean_pooled = cons_mean_pooled.detach().numpy()
test_mean_pooled = test_mean_pooled.detach().numpy()

In [ ]:
# Función para calcular similitud de coseno
def calculate_cosine_similarity(embeddings1, embeddings2):
  for i in range(len(embeddings1)):
    similarities = cosine_similarity(
        [embeddings1[i]],
        embeddings2
    )
    print(f'Array de similitudes: {similarities}')

    # Operaciones para encontrar la similitud más alta y su índice
    max_similarity_index = similarities.argmax()
    max_similarity_value = similarities[0, max_similarity_index]
    print(f"Archivo {i} es más similar a archivo {max_similarity_index} con una similitud de {max_similarity_value}")
    print('---------------------------------------------------------------------')

  return similarities

In [ ]:
# Función para calcular similitud de coseno y determinar plagio
def calculate_plagiarism(test_files, test_embeddings, construction_files, construction_embeddings, threshold=0.9):
    similarity_scores_list = []
    plagiarism_results = []

    for test_file, test_embedding in zip(test_files, test_embeddings):
        similarity_scores = cosine_similarity([test_embedding], construction_embeddings)[0]
        max_similarity = max(similarity_scores)
        percentage_plagiarism = round(max_similarity * 100, 2)

        plagiarized_files = [(os.path.basename(construction_file), score) for score, construction_file in zip(similarity_scores, construction_files) if score >= threshold]

        plagiarism_detected = len(plagiarized_files) > 0
        plagiarism_results.append({
            'Documento sospechoso': os.path.basename(test_file),
            'Copia': 'Sí' if plagiarism_detected else 'No',
            'Documento Plagiado': ', '.join([file for file, score in plagiarized_files]) if plagiarized_files else 'Ninguno',
            '% de Plagio': f'{percentage_plagiarism} %' if plagiarized_files else ''
        })

    return pd.DataFrame(plagiarism_results)

In [ ]:
# Calcular la similitud de coseno entre los archivos de prueba y de construcción
cosine_similarity_result_tc = calculate_cosine_similarity(test_mean_pooled, cons_mean_pooled)

In [ ]:
# Calcular Plagio entre los archivos de prueba y de construcción
plagiarism_df_tc = calculate_plagiarism(test_files, test_mean_pooled, construction_files, cons_mean_pooled)
print(plagiarism_df_tc)

In [ ]:
# Calcular la similitud de coseno entre los archivos de plagio y de construcción
cosine_similarity_result_pc = calculate_cosine_similarity(plag_mean_pooled, cons_mean_pooled)

In [ ]:
# Calcular Plagio entre los archivos de plagio y de construcción
plagiarism_df_pc = calculate_plagiarism(plagiarism_files, plag_mean_pooled, construction_files, cons_mean_pooled)
print(plagiarism_df_pc)

## Evaluación del rendimiento

In [ ]:
# Función para calcular TPR, FPR y AUC
def calculate_metrics(y_true, y_pred, threshold=0.9):
    y_pred_labels = [1 if score >= threshold else 0 for score in y_pred]

    tp = sum((yt == 1 and yp == 1) for yt, yp in zip(y_true, y_pred_labels))
    tn = sum((yt == 0 and yp == 0) for yt, yp in zip(y_true, y_pred_labels))
    fp = sum((yt == 0 and yp == 1) for yt, yp in zip(y_true, y_pred_labels))
    fn = sum((yt == 1 and yp == 0) for yt, yp in zip(y_true, y_pred_labels))

    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    auc = roc_auc_score(y_true, y_pred)

    return tpr, fpr, auc

In [ ]:
# Evaluación del rendimiento con los dataset del profesor
y_true_tc = [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred_tc = [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Calcular métricas
tpr, fpr, auc = calculate_metrics(y_true_tc, y_pred_tc)
print(f'TPR: {tpr}, FPR: {fpr}, AUC: {auc}')

# Parte 2: Clasificación de "Tipo de Plagio"

##Clasificación de archivos

In [ ]:
# Función para obtener la clasificación de un archivo basado en su nombre
def obtener_clasificacion(nombre_archivo):
    numero = int(nombre_archivo.split('-')[-1].split('.')[0])
    if numero >= 1 and numero <= 10:
        return 'Reemplazar frases'
    elif numero >= 11 and numero <= 20:
        return 'Desordenar las frases'
    elif numero >= 21 and numero <= 30:
        return 'Cambio de tiempo'
    elif numero >= 31 and numero <= 40:
        return 'Cambio de voz'
    elif numero >= 41 and numero <= 50:
        return 'Parafraseo'
    elif numero >= 51 and numero <= 60:
        return 'Reemplazar frases'
    elif numero >= 61 and numero <= 70:
        return 'Desordenar las frases'
    elif numero >= 71 and numero <= 80:
        return 'Cambio de tiempo'
    elif numero >= 81 and numero <= 90:
        return 'Cambio de voz'
    elif numero >= 91 and numero <= 100:
        return 'Parafraseo'
    else:
        return None

In [ ]:
# Directorios de los archivos
directorio_original = '/content/drive/MyDrive/RetoDesarrollo/dataset/train'
directorio_plagio = '/content/drive/MyDrive/RetoDesarrollo/dataset/plagiarism'

# Función para leer archivos de un directorio
def leer_archivos(directorio):
    archivos = {}
    for archivo in sorted(os.listdir(directorio)):
        if archivo.endswith('.txt'):
            ruta_archivo = os.path.join(directorio, archivo)
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
            archivos[archivo] = contenido
    return archivos

# Leer archivos de ambos directorios
archivos_originales = leer_archivos(directorio_original)
archivos_plagio = leer_archivos(directorio_plagio)

# Lista para almacenar los datos
datos = []

# Emparejar archivos según el patrón de nombre
for nombre_archivo_plagio in archivos_plagio:
    # Eliminar el prefijo 'not-' del nombre del archivo de plagio para obtener el nombre correspondiente del archivo original
    nombre_archivo_original = nombre_archivo_plagio.replace('not-', '', 1)

    if nombre_archivo_original in archivos_originales:
        contenido_original = archivos_originales[nombre_archivo_original]
        contenido_plagio = archivos_plagio[nombre_archivo_plagio]
        # Obtener el tipo de plagio usando la función obtener_clasificacion
        tipo = obtener_clasificacion(nombre_archivo_plagio)
        datos.append({
            'Contenido_Original': contenido_original,
            'Contenido_Plagio': contenido_plagio,
            'Clasificacion': tipo
        })

# Crear DataFrame
df = pd.DataFrame(datos)

# Mostrar DataFrame
print(df)

In [ ]:
# Dividir el DataFrame en características y etiquetas
X = df.drop(columns=['Clasificacion'])
y = df['Clasificacion']

# Usar train_test_split para dividir el DataFrame en conjuntos de entrenamiento y prueba, asegurando el balanceo de clases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Combinar de nuevo las características y etiquetas para obtener los DataFrames finales
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Mostrar los tamaños de los conjuntos de datos resultantes para verificación
print(f"Tamaño del conjunto de entrenamiento: {train_df.shape}")
print(f"Tamaño del conjunto de prueba: {test_df.shape}")


In [ ]:
#Seleccionar etiquetas únicas dentro de la columna clasificación
print(train_df['Clasificacion'].unique())

In [ ]:
'''
ONE HOT ENCODING PARA TRAIN
'''
# Aplicar One-Hot Encoding a la columna 'Clasificacion'
one_hot_encoded = pd.get_dummies(train_df['Clasificacion'], dtype=int)

# Concatenar el DataFrame original con las columnas One-Hot Encoding
train_df_encoded = pd.concat([train_df, one_hot_encoded], axis=1)

# Eliminar la columna original 'Clasificacion'
train_df_encoded.drop('Clasificacion', axis=1, inplace=True)

# Mostrar el DataFrame resultante con One-Hot Encoding
print(train_df_encoded)

In [ ]:
'''
ONE HOT ENCODING PARA TEST
'''
# Aplicar One-Hot Encoding a la columna 'Clasificacion'
one_hot_encoded = pd.get_dummies(test_df['Clasificacion'], dtype=int)

# Concatenar el DataFrame original con las columnas One-Hot Encoding
test_df_encoded = pd.concat([test_df, one_hot_encoded], axis=1)

# Eliminar la columna original 'Clasificacion'
test_df_encoded.drop('Clasificacion', axis=1, inplace=True)

# Mostrar el DataFrame resultante con One-Hot Encoding
print(test_df_encoded)

In [ ]:
column_labels = train_df_encoded.columns.tolist()[2:]
label_counts = train_df_encoded[column_labels].sum().sort_values()

# Crear un fondo para la gráfica
plt.figure(figsize=(7, 5))

# Crear un gráfico de barras horizontal usando Seaborn
ax = sns.barplot(x=label_counts.values,
                 y=label_counts.index, palette='viridis')

# Añadir etiquetas y título a la gráfica
plt.xlabel('Número de Ocurrencias')
plt.ylabel('Etiquetas')
plt.title('Distribución de las Ocurrencias de Etiquetas')

# Mostrar la gráfica
plt.show()

In [ ]:
train_df_encoded[column_labels].sum().sort_values()

##Embedding

In [ ]:
# Función para obtener el embedding de BERT de un texto
def get_bert_embedding(text):
    # Obtener el embedding de BERT
    embedding = model.encode(text)
    return embedding

In [ ]:
# Cargar el modelo SentenceTransformer pre-entrenado
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# Aplicar la función de limpieza de texto a cada fila del DataFrame
train_df_encoded['Contenido_Original_Limpio'] = clean_text(train_df_encoded['Contenido_Original'])
train_df_encoded['Contenido_Plagio_Limpio'] = clean_text(train_df_encoded['Contenido_Plagio'])

# Aplicar la función de embedding de BERT a cada fila del DataFrame limpio
train_df_encoded['Embedding_Original'] = train_df_encoded['Contenido_Original_Limpio'].apply(lambda x: get_bert_embedding(x))
train_df_encoded['Embedding_Plagiado'] = train_df_encoded['Contenido_Plagio_Limpio'].apply(lambda x: get_bert_embedding(x))

In [ ]:
# Crear un nuevo DataFrame con los embeddings y las otras características
embedding_train_df = train_df_encoded[['Embedding_Original', 'Embedding_Plagiado', 'Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']]

# Mostrar el nuevo DataFrame
print(embedding_train_df.head())

In [ ]:
# Aplicar la función de limpieza de texto a cada fila del DataFrame
test_df_encoded['Contenido_Original_Limpio'] = clean_text(test_df_encoded['Contenido_Original'])
test_df_encoded['Contenido_Plagio_Limpio'] = clean_text(test_df_encoded['Contenido_Plagio'])

# Aplicar la función de embedding de BERT a cada fila del DataFrame limpio
test_df_encoded['Embedding_Original'] = test_df_encoded['Contenido_Original_Limpio'].apply(lambda x: get_bert_embedding(x))
test_df_encoded['Embedding_Plagiado'] = test_df_encoded['Contenido_Plagio_Limpio'].apply(lambda x: get_bert_embedding(x))

In [ ]:
# Crear un nuevo DataFrame con los embeddings y las otras características
embedding_test_df = test_df_encoded[['Embedding_Original', 'Embedding_Plagiado', 'Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']]

# Mostrar el nuevo DataFrame
print(embedding_test_df.head())

In [ ]:
# Asignar el DataFrame de entrenamiento codificado a la variable 'train_data'
train_data = embedding_train_df

# Asignar el DataFrame de prueba codificado a la variable 'test_data'
test_data = embedding_test_df

# Obtener las etiquetas (nombres de columnas) a partir de la tercera columna del DataFrame de entrenamiento
train_labels = train_data.columns[2:]

# Obtener las etiquetas (nombres de columnas) a partir de la tercera columna del DataFrame de prueba
test_labels = train_data.columns[2:]

# Imprimir las etiquetas de entrenamiento
print(train_labels)

# Imprimir las etiquetas de prueba
print(test_labels)


In [ ]:
print(train_data.shape)
print(train_labels.shape)
print('----------------------------')
print(test_data.shape)
print(test_labels.shape)

##Tokenizar

In [ ]:
# Función para tokenizar y codificar
def tokenize_and_encode(tokenizer, plagiarized, original, labels, max_length=128):
	# Inicializar listas vacías para almacenar las entradas tokenizadas y las máscaras de atención
	input_ids = []
	attention_masks = []

	# Iterar a través de cada comentario en las listas 'plagiarized' y 'original'
	for plagiarized, original in zip(plagiarized, original):

		# Tokenizar y codificar el comentario utilizando el tokenizador de BERT
		encoded_dict = tokenizer.encode_plus(
			plagiarized,
			text_pair=original,
			# Añadir tokens especiales como [CLS] y [SEP]
			add_special_tokens=True,

			# Truncar o rellenar el comentario a 'max_length'
			max_length=max_length,

			truncation=True,

			# Rellenar el comentario hasta 'max_length' con ceros si es necesario
			pad_to_max_length=True,

			# Devolver la máscara de atención para enmascarar los tokens rellenados
			return_attention_mask=True,

			# Devolver tensores de PyTorch
			return_tensors='pt'
		)

		# Añadir la entrada tokenizada y la máscara de atención a sus respectivas listas
		input_ids.append(encoded_dict['input_ids'])
		attention_masks.append(encoded_dict['attention_mask'])

	# Concatenar las entradas tokenizadas y las máscaras de atención en tensores
	input_ids = torch.cat(input_ids, dim=0)
	attention_masks = torch.cat(attention_masks, dim=0)

	# Convertir las etiquetas a un tensor de PyTorch con el tipo de dato float32
	labels = torch.tensor(labels, dtype=torch.float32)

	# Devolver las entradas tokenizadas, las máscaras de atención y las etiquetas como tensores de PyTorch
	return input_ids, attention_masks, labels


In [ ]:
# Inicialización del Tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
										do_lower_case=True)

In [ ]:
# Inicialización del Modelo
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
													num_labels=5)


In [ ]:
# Mover el modelo a GPU si está disponible
device = torch.device(
	'cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
train_data = train_df_encoded
test_data = test_df_encoded

In [ ]:
# Verifica las columnas de los DataFrames
print("Train Data Columns: ", train_data.columns)
print("Test Data Columns: ", test_data.columns)

In [ ]:
print(train_data)
print(test_data)

In [ ]:
# Extraer comentarios y etiquetas del DataFrame de datos de entrenamiento
plagiarized = train_data['Contenido_Plagio_Limpio'].tolist()
original = train_data['Contenido_Original_Limpio'].tolist()
plagiarized_test = test_data['Contenido_Plagio_Limpio'].tolist()
original_test = test_data['Contenido_Original_Limpio'].tolist()
train_labels = train_data[['Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']].values
test_labels = test_data[['Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']].values

In [ ]:
# Tokenizar y codificar los comentarios y etiquetas para el conjunto de entrenamiento
input_ids, attention_masks, train_labels = tokenize_and_encode(
	tokenizer,
	plagiarized,
	original,
	train_labels,
	max_length=128
)

# Tokenizar y codificar los comentarios y etiquetas para el conjunto de prueba
test_input_ids, test_attention_masks, test_labels = tokenize_and_encode(
	tokenizer,
	plagiarized_test,
	original_test,
	test_labels,
	max_length=128
)

# Revisar dimensiones de los tensores
print('Textos de Entrenamiento:', train_data.shape)
print('ID de Entrada:', input_ids.shape)
print('Máscara de Atención:', attention_masks.shape)
print('Etiquetas de Entrenamiento:', train_labels.shape)
print('---------------------------------------------------------')
print('Textos de Prueba:', test_data.shape)
print('ID de Entrada de Prueba:', test_input_ids.shape)
print('Máscara de Atención de Prueba:', test_attention_masks.shape)
print('Etiquetas de Prueba:', test_labels.shape)

##Entrenamiento del Modelo

In [ ]:
k = 6
print('Textos de Entrenamiento -->>', train_data.values[k])
print('\nID de Entrada -->>\n', input_ids[k])
print('\nID Decodificado -->>\n', tokenizer.decode(input_ids[k]))
print('\nMáscara de Atención -->>\n', attention_masks[k])
print('\nEtiquetas -->>', train_labels[[k]])

In [ ]:
# Verifica las columnas de los DataFrames
print(train_data.columns)
print(test_data.columns)

In [ ]:
train_df=train_data[['Embedding_Original', 'Embedding_Plagiado', 'Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']]
test_df=test_data[['Embedding_Original', 'Embedding_Plagiado', 'Cambio de tiempo', 'Cambio de voz', 'Desordenar las frases', 'Parafraseo', 'Reemplazar frases']]

In [ ]:
# Verifica los DataFrames
print(train_df)
print(test_df)

In [ ]:
print(f"train_labels: {train_labels}")
print(f"Length of train_labels: {len(train_labels)}")

In [ ]:
# Creación de DataLoader para el conjunto de datos de entrenamiento balanceado
batch_size = 32
train_dataset = TensorDataset(input_ids, attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Conjunto de prueba
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
print('Tamaño del Lote:', train_loader.batch_size)
batch = next(iter(train_loader))
print('Forma de cada ID de Entrada:', batch[0].shape)
print('IDs de Entrada:\n', batch[0][0])
print('Texto Decodificado Correspondiente:\n', tokenizer.decode(batch[0][0]))
print('Máscara de Atención Correspondiente:\n', batch[1][0])
print('Etiqueta Correspondiente:', batch[2][0])

In [ ]:
# Configuración del Optimizador
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
# Inicialización del Modelo
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
													num_labels=5)

In [ ]:
def train_model(model, train_loader, optimizer, device, num_epochs):
    # Iterar a través del número especificado de épocas
    for epoch in range(num_epochs):
        # Establecer el modelo en modo de entrenamiento
        model.train()
        # Inicializar la pérdida total para la época actual
        total_loss = 0

        # Iterar a través de los lotes en los datos de entrenamiento
        for batch in train_loader:
          # Imprimir el lote actual para depuración
          print(batch)
          # Mover los tensores del lote al dispositivo especificado (CPU o GPU)
          input_ids, attention_mask, labels = [t.to(device) for t in batch]

          # Reiniciar los gradientes acumulados en el optimizador
          optimizer.zero_grad()

          # Propagar hacia adelante: calcular la salida y la pérdida del modelo
          outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss
          total_loss += loss.item()

          # Propagar hacia atrás: calcular los gradientes de la pérdida con respecto a los parámetros del modelo
          loss.backward()
          # Actualizar los parámetros del modelo utilizando el optimizador
          optimizer.step()

        # Imprimir la pérdida promedio para la época actual
        print(f'Época {epoch+1}, Pérdida de Entrenamiento: {total_loss/len(train_loader)}')

# Crear DataLoader para el conjunto de datos balanceado
tamaño_lote = 32
conjunto_datos_entrenamiento = TensorDataset(input_ids, attention_masks, train_labels)
cargador_entrenamiento = DataLoader(conjunto_datos_entrenamiento, batch_size=tamaño_lote, shuffle=True)

# Conjunto de prueba
conjunto_datos_prueba = TensorDataset(test_input_ids, test_attention_masks, test_labels)
cargador_prueba = DataLoader(conjunto_datos_prueba, batch_size=tamaño_lote, shuffle=False)

# Llamar a la función para entrenar el modelo
train_model(model, train_loader, optimizer, device, num_epochs=3)


##Evaluar Modelo

In [ ]:
# Evaluar el Modelo
def evaluate_model(model, test_loader, device):
	model.eval() # Establecer el modelo en modo de evaluación

	true_labels = []
	predicted_probs = []

	with torch.no_grad():
		for batch in test_loader:
			input_ids, attention_mask, labels = [t.to(device) for t in batch]

			# Obtener las predicciones del modelo
			outputs = model(input_ids, attention_mask=attention_mask)
			# Utilizar la función sigmoide para clasificación multietiqueta
			predicted_probs_batch = torch.sigmoid(outputs.logits)
			predicted_probs.append(predicted_probs_batch.cpu().numpy())

			true_labels_batch = labels.cpu().numpy()
			true_labels.append(true_labels_batch)

	# Combinar predicciones y etiquetas para evaluación
	true_labels = np.concatenate(true_labels, axis=0)
	predicted_probs = np.concatenate(predicted_probs, axis=0)
	predicted_labels = (predicted_probs > 0.5).astype(
		int) # Aplicar umbral para clasificación binaria

	# Calcular métricas de evaluación
	accuracy = accuracy_score(true_labels, predicted_labels)
	precision = precision_score(true_labels, predicted_labels, average='micro')
	recall = recall_score(true_labels, predicted_labels, average='micro')

	# Imprimir las métricas de evaluación
	print(f'Precisión: {accuracy:.4f}')
	print(f'Precision: {precision:.4f}')
	print(f'Recall: {recall:.4f}')


# Llamar a la función para evaluar el modelo en los datos de prueba
evaluate_model(model, test_loader, device)

In [ ]:
# Directorio de salida para guardar el modelo y el tokenizer
output_dir = "Saved_model"
# Guardar el diccionario de estado y la configuración del modelo
model.save_pretrained(output_dir)
# Guardar la configuración y el vocabulario del tokenizer
tokenizer.save_pretrained(output_dir)

In [ ]:
# Nombre del directorio donde se guardaron el modelo y el tokenizer
model_name = "Saved_model"
# Cargar el tokenizer desde el directorio guardado
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
# Cargar el modelo desde el directorio guardado y moverlo al dispositivo adecuado (CPU o GPU)
Bert_Model = BertForSequenceClassification.from_pretrained(
	model_name).to(device)

In [ ]:
def classify_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer, device=device):
    # Convertir el texto de entrada en una lista
    input = [input_text]

    # Codificar el texto de entrada utilizando el tokenizer
    encodings = tokenizer(
        input, truncation=True, padding=True, return_tensors="pt")

    # Crear un conjunto de datos TensorDataset para el texto de entrada
    dataset = TensorDataset(
        encodings['input_ids'], encodings['attention_mask'])

    # Crear un DataLoader para el conjunto de datos
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    # Establecer el modelo en modo de evaluación
    model.eval()

    # Realizar la clasificación del texto de entrada
    with torch.no_grad():
        for batch in loader:
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)

    # Convertir las predicciones en etiquetas binarias utilizando un umbral de 0.5
    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)
    labels_list = ['Time Change', 'Voice Change',
                   'Sentence Rearrangement', 'Paraphrasing', 'Phrase Replacement']
    result = dict(zip(labels_list, predicted_labels[0]))
    return result


# Texto de entrada para clasificar
text = 'Interactive software agents, such as chatbots, are progressively being used in the area of health and well-being. In such applications, where agents engage with users in interpersonal conversations for, e.g., coaching, comfort or behavior-change interventions, there is an increased need for understanding agents’ empathic capabilities. In the current state-of-the-art, there are no tools to do that. In order to understand empathic capabilities in interactive software agents, we need a precise notion of empathy. The literature discusses a variety of definitions of empathy, but there is no consensus of a formal definition. Based on a systematic literature review and a qualitative analysis of recent approaches to empathy in interactive agents for health and well-being, a formal definition—an ontology—of empathy is developed. We present the potential of the formal definition in a controlled user-study by applying it as a tool for assessing empathy in two state-of-the-art health and well-being chatbots; Replika and Wysa. Interactive software agents, such as chatbots, are progressively being used in the area of health and well-being. In such applications, where agents engage with users in interpersonal conversations for, e.g., coaching, comfort or behavior-change interventions, there is an increased need for understanding agents’ empathic capabilities. In the current state-of-the-art, there are no tools to do that. In order to understand empathic capabilities in interactive software agents, we need a precise notion of empathy. The literature discusses a variety of definitions of empathy, but there is no consensus of a formal definition. We present the potential of the formal definition in a controlled user-study by applying it as a tool for assessing empathy in two state-of-the-art health and well-being chatbots; Replika and Wysa. Our findings suggest that our definition captures necessary conditions for assessing empathy in interactive agents, and how it can uncover and explain trends in changing perceptions of empathy over time. The definition, implemented in Web Ontology Language (OWL), may serve as an automated tool, enabling systems to recognize empathy in interactions—be it an interactive agent evaluating its own empathic performance or an intelligent system assessing the empathic capability of its interlocutors.'
classify_input(input_text=tex
